In [2]:
import sqlite3 as sq
import pandas as pd
from pandas.io import sql
from datetime import datetime, timedelta
from pytz import timezone, all_timezones
from random import randint
import uuid
Base='C:/Users/Jyotindra/Desktop/DSPracs'
print('Working Base :',Base)
sInputFileName='/Assess_People.csv'
sDataBaseDir=Base
sDatabaseName=sDataBaseDir + '/clark.db'
conn1 = sq.connect(sDatabaseName)

sDatabaseName=Base + '/datavault.db'
conn2 = sq.connect(sDatabaseName)
### Import Female Data
sFileName=Base + sInputFileName
print('Loading :',sFileName)
print('################################')
print(sFileName)
RawData=pd.read_csv(sFileName,header=0,low_memory=False, encoding="latin-1")
RawData.drop_duplicates(subset=None, keep='first', inplace=True)
start_date = datetime(1900,1,1,0,0,0)
start_date_utc=start_date.replace(tzinfo=timezone('UTC'))
HoursBirth=100*365*24
RawData['BirthDateUTC']=RawData.apply(lambda row:
(start_date_utc + timedelta(hours=randint(0, HoursBirth)))
,axis=1)
zonemax=len(all_timezones)-1
RawData['TimeZone']=RawData.apply(lambda row:
(all_timezones[randint(0, zonemax)])
,axis=1)
RawData['BirthDateISO']=RawData.apply(lambda row:
row["BirthDateUTC"].astimezone(timezone(row['TimeZone']))
,axis=1)
RawData['BirthDateKey']=RawData.apply(lambda row:
row["BirthDateUTC"].strftime("%Y-%m-%d %H:%M:%S")
,axis=1)
RawData['BirthDate']=RawData.apply(lambda row:
row["BirthDateISO"].strftime("%Y-%m-%d %H:%M:%S")
,axis=1)
RawData['PersonID']=RawData.apply(lambda row:
str(uuid.uuid4())
,axis=1)
Data=RawData.copy()
Data.drop('BirthDateUTC', axis=1,inplace=True)
Data.drop('BirthDateISO', axis=1,inplace=True)
indexed_data = Data.set_index(['PersonID'])
print('################################')
sTable='Process_Person'
print('Storing :',sDatabaseName,' Table:',sTable)
indexed_data.to_sql(sTable, conn1, if_exists="replace")
PersonHubRaw=Data[['PersonID','FirstName','SecondName','LastName','BirthDateKey']]
PersonHubRaw['PersonHubID']=RawData.apply(lambda row:
str(uuid.uuid4())
,axis=1)
PersonHub=PersonHubRaw.drop_duplicates(subset=None, \
keep='first',\
inplace=False)
indexed_PersonHub = PersonHub.set_index(['PersonHubID'])
sTable = 'Hub-Person'
print('Storing :',sDatabaseName,' Table:',sTable)
indexed_PersonHub.to_sql(sTable, conn2, if_exists="replace")
PersonSatelliteGenderRaw=Data[['PersonID','FirstName','SecondName','LastName'\
,'BirthDateKey','Gender']]
PersonSatelliteGenderRaw['PersonSatelliteID']=RawData.apply(lambda row:
str(uuid.uuid4())
,axis=1)
PersonSatelliteGender=PersonSatelliteGenderRaw.drop_duplicates(subset=None, \
keep='first', \
inplace=False)
indexed_PersonSatelliteGender = PersonSatelliteGender.set_index(['PersonSatelliteID'])
sTable = 'Satellite-Person-Gender'
print('Storing :',sDatabaseName,' Table:',sTable)
indexed_PersonSatelliteGender.to_sql(sTable, conn2, if_exists="replace")
################################################################
PersonSatelliteBirthdayRaw=Data[['PersonID','FirstName','SecondName','LastName',\
'BirthDateKey','TimeZone','BirthDate']]
PersonSatelliteBirthdayRaw['PersonSatelliteID']=RawData.apply(lambda row:
str(uuid.uuid4())
,axis=1)
PersonSatelliteBirthday=PersonSatelliteBirthdayRaw.drop_duplicates(subset=None, \
keep='first',\
inplace=False)
indexed_PersonSatelliteBirthday = PersonSatelliteBirthday.set_index(['PersonSatelliteID'])
sTable = 'Satellite-Person-Names'
print('Storing :',sDatabaseName,' Table:',sTable)
indexed_PersonSatelliteBirthday.to_sql(sTable, conn2, if_exists="replace")
################################################################
sFileDir=Base
sOutputFileName = sTable + '.csv'
sFileName=sFileDir + '/' + sOutputFileName
print('Storing :', sFileName)
print('################################')
RawData.to_csv(sFileName, index = False)
print('Vacuum Databases')
sSQL="VACUUM;"
sql.execute(sSQL,conn1)
sql.execute(sSQL,conn2)
print('Done!!')
print('Nishi Jain-53004230036')

Working Base : C:/Users/Jyotindra/Desktop/DSPracs
Loading : C:/Users/Jyotindra/Desktop/DSPracs/Assess_People.csv
################################
C:/Users/Jyotindra/Desktop/DSPracs/Assess_People.csv
################################
Storing : C:/Users/Jyotindra/Desktop/DSPracs/datavault.db  Table: Process_Person


C:\Users\Jyotindra\AppData\Local\Temp\ipykernel_10276\2071349370.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PersonHubRaw['PersonHubID']=RawData.apply(lambda row:


Storing : C:/Users/Jyotindra/Desktop/DSPracs/datavault.db  Table: Hub-Person


C:\Users\Jyotindra\AppData\Local\Temp\ipykernel_10276\2071349370.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PersonSatelliteGenderRaw['PersonSatelliteID']=RawData.apply(lambda row:


Storing : C:/Users/Jyotindra/Desktop/DSPracs/datavault.db  Table: Satellite-Person-Gender
Storing : C:/Users/Jyotindra/Desktop/DSPracs/datavault.db  Table: Satellite-Person-Names
Storing : C:/Users/Jyotindra/Desktop/DSPracs/Satellite-Person-Names.csv
################################
Vacuum Databases


C:\Users\Jyotindra\AppData\Local\Temp\ipykernel_10276\2071349370.py:99: FutureWarning: `pandas.io.sql.execute` is deprecated and will be removed in the future version.
  sql.execute(sSQL,conn1)
C:\Users\Jyotindra\AppData\Local\Temp\ipykernel_10276\2071349370.py:100: FutureWarning: `pandas.io.sql.execute` is deprecated and will be removed in the future version.
  sql.execute(sSQL,conn2)


Done!!
Nishi Jain-53004230036
